In [1]:
from exp.nb_03 import *
from torch import optim

In [55]:
x_train,y_train,x_valid,y_valid=get_data()

In [56]:
# Let say I don't normalize my input

In [57]:
train_ds,valid_ds=Dataset(x_train,y_train),Dataset(x_valid,y_valid)

In [58]:
train_dl,valid_dl=get_dls(train_ds,valid_ds,bs=64)

In [59]:
nh=50
c=y_train.max()+1
ni=x_train.shape[1]
ni,c,nh

(784, tensor(10), 50)

In [60]:
epochs = 10


In [104]:
class MyModel(nn.Module):
    def __init__(self,ni,nh,c):
        super().__init__()
        self.ni=ni
        self.nh=nh
        self.c=c
        self.layers=nn.ModuleList([nn.Linear(ni,nh),nn.BatchNorm1d(nh),nn.ReLU(),nn.BatchNorm1d(nh),nn.Linear(nh,10)])
    def forward(self,x):
        for l in self.layers:
            x=l(x)
           ## print(f"After layer => {l},mean=>{x.mean()},std=>{x.std()}")
        return x

        
      

In [105]:
model=MyModel(ni,nh,c)

In [106]:
opt=optim.SGD(model.parameters(),lr=0.01)

In [107]:
for epoch in range(epochs):
    for xb,yb in train_dl:
        pred = model(xb)
        loss = F.cross_entropy(pred,yb)
        loss.backward()
        opt.step()
        opt.zero_grad
    print(f"Loss at{epoch} ==> {loss}")

Loss at0 ==> 2.0353126525878906
Loss at1 ==> 10.691108703613281
Loss at2 ==> 42.754005432128906
Loss at3 ==> 394.425537109375
Loss at4 ==> 813.5791625976562
Loss at5 ==> 33276.375
Loss at6 ==> 40714.3203125
Loss at7 ==> 495217.75
Loss at8 ==> 1725694.75
Loss at9 ==> 33495224.0


In [46]:
nn.BatchNorm1d??

In [69]:
#say now we normalize

In [128]:
x_train,y_train,x_valid,y_valid=get_data()
mean=x_train.mean()
std=x_train.std()
x_train=normalize(x_train,mean,std)
x_valid=normalize(x_valid,mean,std)

In [129]:
train_ds,valid_ds=Dataset(x_train,y_train),Dataset(x_valid,y_valid)

In [130]:
train_dl,valid_dl=get_dls(train_ds,valid_ds,bs=64)

In [150]:
class MyModel(nn.Module):
    def __init__(self,ni,nh,c):
        super().__init__()
        self.ni=ni
        self.nh=nh
        self.c=c
        self.layers=nn.ModuleList([nn.Linear(ni,nh),nn.BatchNorm1d(nh),nn.ReLU(),nn.Linear(nh,10)])
    def forward(self,x):
        for l in self.layers:
            x=l(x)
            print(f"After layer => {l},mean=>{x.mean()},std=>{x.std()}")
        return x


In [160]:
model=nn.Sequential(
    nn.Linear(ni,nh),
    nn.ReLU(),
    nn.BatchNorm1d(nh),
    nn.Linear(nh,10)
)

In [161]:
for epoch in range(epochs):
    for xb,yb in train_dl:
        pred = model(xb)
        loss = F.cross_entropy(pred,yb)
        loss.backward()
        opt.step()
        opt.zero_grad

In [162]:
xb,yb=next(iter(train_dl))

In [163]:
loss=F.cross_entropy(model(xb),yb)

In [164]:
loss

tensor(2.3446, grad_fn=<NllLossBackward>)

In [136]:
xb.mean(),xb.std()

(tensor(0.0126), tensor(1.0155))

In [167]:
from fastai import data_block
from fastai import *

In [2]:
class BatchNorm(nn.Module):
    def __init__(self,nf,mom=0.9,eps=0.01):
        super().__init__()
        self.nf=nf
        self.mom=mom
        self.eps=eps
        self.mults=nn.Parameter(torch.ones(nf,1,1))
        self.adds=nn.Parameter(torch.zeros(nf,1,1))
        self.register_buffer('vars',torch.ones(1,nf,1,1))
        self.register_buffer('means',torch.zeroes(1,nf,1,1))
    def update_stats(x):
        m=x.mean((0,2,3),keepdim=True)
        v=x.var((0,2,3),keepdim=True)
        self.means.lerp_(m,self.mom)
        self.means.lerp_(v,self.mom)
    def forward(self,x):
        if self.training:
            with torch.no_grad():      
                m,v=self.update_stats(x)
        else:
            m,v=self.means,self.vars
        x=(x-m)/(v+self.eps).sqrt()
        return self.mults*x+self.adds